In [24]:
import numpy as np
import json
from scipy import misc
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from matplotlib import pylab as plt
%matplotlib inline

/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [26]:
train_data = json.load(open('train.json','r'))

In [8]:
all_ingredients = []
all_dishes = []
for dish in train_data:
    for ingredient in dish[u'ingredients']:
        all_ingredients.append(ingredient.strip())
    all_dishes.append(dish[u'cuisine'].strip())
print('Number of dishes: '+str(len(train_data)))
print('Number of unique ingredients: '+str(len(set(all_ingredients))))
print('Number of cuisines: '+str(len(set(all_dishes))))

Number of dishes: 39774
Number of ingredients: 6714
Number of cuisines: 20


In [14]:
#(c) binary encoding of ingredients list
ing_dict=dict()
ind = 0
for i in set(all_ingredients):
    ing_dict[i]=ind
    ind+=1

In [21]:
train_binary = []
for dish in train_data:
    dish_binary = [0]*len(set(all_ingredients))
    for ingredient in dish[u'ingredients']:
        dish_binary[ing_dict[ingredient]]=1
    train_binary.append(dish_binary)
train_binary = np.array(train_binary, dtype=int)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500


In [22]:
#(d)
train_label = np.array(all_dishes, dtype = str)

In [25]:
nb_gaussian = GaussianNB()
nb_bernoulli = BernoulliNB()
nb_logis = LogisticRegression()

crossval = KFold(len(train_binary), n_folds=3)
avg_gaussian, avg_bernoulli, avg_logis =0,0,0

for tr_ind, te_ind in crossval:
    xtr= train_binary[tr_ind]
    xte = train_binary[te_ind]
    ytr = train_label[tr_ind]
    yte = train_label[te_ind]
    
    nb_gaussian.fit(xtr, ytr)
    nb_bernoulli.fit(xtr, ytr)
    nb_logis.fit(xtr, ytr)
    avg_gaussian += nb_gaussian.score(xte, yte)/3
    avg_bernoulli += nb_bernoulli.score(xte, yte)/3
    avg_logis += nb_logis.score(xte, yte)/3

print("Gaussian: "+str(avg_gaussian))
print("Bernoulli: "+str(avg_bernoulli))
print("Logistic Regression: "+str(avg_logis))

Gaussian: 0.379846130638
Bernoulli: 0.683536983959
Logistic Regression: 0.775556896465


In [29]:
test = json.load(open('test.json','r'))
test_data = []
test_id = []
for dish in test:
    test_id.append(dish[u'id'])
    dish_binary = [0]*len(set(all_ingredients))
    for ingredient in dish[u'ingredients']:
        if ingredient in ing_dict:
            dish_binary[ing_dict[ingredient]]=1
    test_data.append(dish_binary)
test_data = np.array(test_data, dtype=int)
test_id = np.array(test_id)
test_pred = nb_logis.predict(test_data)


TypeError: Mismatch between array dtype ('|S21') and format specifier ('%.18e %.18e')

In [34]:
np.savetxt('cooking_handin.csv', np.column_stack([test_id,test_pred]),delimiter=',',fmt='%s')